In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import login

# ✅ Login if needed (Hugging Face API token)
login()

# Load model and tokenizer for the medical expert LLM
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto", use_auth_token=True)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Function to assess question quality
def evaluate_question_quality(dialogue):
    prompt = f"""
    You are a senior medical expert. Based on the following patient-doctor dialogue, rate the quality of the questions asked:
    "{dialogue}"
    The quality of the questions should be evaluated based on their clarity, relevance, and how well they help in diagnosing the patient.
    Provide a score from 1 (poor) to 10 (excellent) and a brief explanation.
    """
    response = llm(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
    return response.strip()

# Function to assess the correctness of the assigned specialist
def evaluate_specialist(dialogue, assigned_specialist):
    prompt = f"""
    You are a senior medical expert. Given the following patient-doctor dialogue, assess if the assigned specialist is correct:
    Dialogue: "{dialogue}"
    Assigned Specialist: {assigned_specialist}
    Based on the patient's condition described in the dialogue, is this the correct specialist to be assigned? Answer with "Correct" or "Incorrect" and provide a brief explanation.
    """
    response = llm(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']
    return response.strip()

# Function to compute evaluation score for each case
def evaluate_case(row):
    dialogue = row['dialogue']
    assigned_specialist = row['specialist_label']
    
    # Evaluate the question quality and specialist correctness
    question_quality = evaluate_question_quality(dialogue)
    specialist_evaluation = evaluate_specialist(dialogue, assigned_specialist)
    
    # Generate a score for each
    question_score = int(question_quality.split()[0]) if question_quality.split()[0].isdigit() else 0
    specialist_score = 1 if "Correct" in specialist_evaluation else 0
    
    # Return the results
    return question_score, specialist_score, question_quality, specialist_evaluation

# Load the dataset containing the cases (only first 30 rows)
df = pd.read_csv("/teamspace/studios/this_studio/labeled_data.csv").head(30)

# Apply the evaluation function
df[['Question Quality Score', 'Specialist Score', 'Question Quality Explanation', 'Specialist Evaluation']] = df.apply(evaluate_case, axis=1, result_type="expand")

# Calculate overall accuracy for specialists (using predicted labels vs. evaluated correctness)
accuracy = df['Specialist Score'].mean() * 100

# Calculate overall question quality score
average_question_quality = df['Question Quality Score'].mean()

# Save the results to a new CSV
# df.to_csv("/teamspace/studios/this_studio/evaluated_results_with_question_quality_first_20.csv", index=False)

# Output evaluation scores
print(f"✅ Evaluation complete for the first 30 rows. Overall specialist accuracy: {accuracy:.2f}%")
print(f"✅ Average question quality score: {average_question_quality:.2f}")



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to

✅ Evaluation complete for the first 30 rows. Overall specialist accuracy: 65
✅ Average question quality score: 6 
